Поскольку мы агрегировали данные по равномерной сетке, можно ожидать, что существенное количество ячеек окажется в районах города, где вызов такси невозможен (реки, заливы, острова, парки). Теоретически ряды, соответствующие таким ячейка, должны состоять из одних нулей, но на практике это может быть не так: устройства, определяющие координаты автомобилей в момент начала поездки, несовершенны и могут ошибаться, показывая, например, что такси находится в середине реки. Чтобы правильно отобрать ячейки, вам понадобится визуализировать их границы на карте и отфильтровать те, из которых поездки на такси невозможны. 

# Загрузите агрегированные данные о поездках в мае 2016. Просуммируйте общее количество поездок такси из каждой географической зоны и посчитайте количество ячеек, из которых в мае не было совершено ни одной поездки.

In [1]:
import pandas as pd
taxi201605_agg = pd.read_csv("may2016_aggregated.csv")

In [2]:
taxi201605_agg

,Unnamed: 0,1,2,3,4,5,6,7,8,9,...,2491,2492,2493,2494,2495,2496,2497,2498,2499,2500
0,2016-05-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2016-05-01 01:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2016-05-01 02:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2016-05-01 03:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2016-05-01 04:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,2016-05-31 19:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
740,2016-05-31 20:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
741,2016-05-31 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
742,2016-05-31 22:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Просуммируйте общее количество поездок такси из каждой географической зоны

In [3]:
pickups = taxi201605_agg.sum(axis=0)
pickups[1:]

1       0
2       0
3       0
4       0
5       0
       ..
2496    0
2497    0
2498    0
2499    0
2500    0
Length: 2500, dtype: object

In [4]:
pickups[1231]

489489.0

количество ячеек, из которых в мае не было совершено ни одной поездки

In [5]:
(pickups[1:]<=0).sum()

1283

удаляем пустые области

In [6]:
pickups = pickups[1:][pickups[1:]>0]

# Нарисуйте статическую карту Нью-Йорка. Поставьте на карте точку там, где находится Эмпайр-Стейт-Билдинг.

In [7]:
# ESB
lat, lon = 40.748817, -73.985428

## [follium](https://python-visualization.github.io/folium/quickstart.html)

In [8]:
from folium import Map, Marker
m = Map(location=(lat, lon), zoom_start=10, no_touch=True, zoom_control=False,
        draggable=False, touch_zoom=False, double_click_zoom=False)
marker = Marker(location=(lat, lon), draggable=False,
                title='Empire State Building')
m.add_child(marker)
m

# Поверх статической карты Нью-Йорка визуализируйте данные о поездках из каждой ячейки так, чтобы цветовая шкала, в которую вы окрашиваете каждую ячейку, показывала суммарное количество поездок такси из неё.

In [9]:
import numpy as np

функция определения границ областей

In [10]:
def region_edges(i: int) -> list:
    m = 50
    # edges
    w = -74.25559
    e = -73.70001
    n = 40.91553
    s = 40.49612
    x_step = (e-w)/m
    y_step = (n-s)/m
    x = np.ceil(i/m)-1
    y = (i-1) % m
    return [(w+x*x_step, s+y*y_step),
            (w+x*x_step, s+y*y_step+y_step),
            (w+x*x_step+x_step, s+y*y_step+y_step),
            (w+x*x_step+x_step, s+y*y_step),
            ]

In [11]:
# проверка
i = 1231
print(region_edges(i))
r = pd.read_csv("regions.csv", sep = ";")
r[r.region == i]

[(-73.98891160000001, 40.747766), (-73.98891160000001, 40.7561542), (-73.9778, 40.7561542), (-73.9778, 40.747766)]


,region,west,east,south,north
1230,1231,-73.988912,-73.9778,40.747766,40.756154


генерация полигонов

In [12]:
state_geo = {'type': "FeatureCollection",
             'features': []}

for i in range(1, 2501, 1):
    region_geo_data = {"type": "Feature",
                      "id": str(i),
                      "geometry": {
                          "type": "Polygon",
                          "coordinates": [
                              region_edges(i)
                          ]
                      },
#                       "properties": {
#                           "name": "New York",
#                       }
                      }
    state_geo['features'].append(region_geo_data)

In [13]:
state_data = pd.DataFrame()
state_data["id"]=pickups[1:].index.values
state_data["counts"] = pickups[1:].values

In [14]:
# проверка
state_data[state_data.id=="1231"]

,id,counts
454,1231,489489


In [15]:
from folium import Choropleth, CircleMarker

m = Map(location=(lat, lon), zoom_start=10, no_touch=False, zoom_control=False,
        draggable=False, touch_zoom=False, double_click_zoom=False)

Choropleth(
    geo_data=state_geo,
    name='taxi',
    data=state_data,
    columns=['id', 'counts'],
    key_on='feature.id',
    fill_color='OrRd',
    fill_opacity=0.6,
    line_opacity=0.2,
).add_to(m)

m

# Вставьте интерактивную карту Нью-Йорка — такую, которую можно прокручивать и увеличивать. Поставьте метку там, где находится [статуя свободы](https://www.latlong.net/place/the-statue-of-liberty-national-monument-new-york-usa-2122.html).

Latitude and longitude coordinates are: 40.689247, -74.044502.

In [16]:
# sol
lat, lon = 40.689247, -74.044502
m = Map(location=(lat, lon), zoom_start=10, no_touch=False, zoom_control=True,
        draggable=True, touch_zoom=True, double_click_zoom=True)
marker = Marker(location=(lat, lon), draggable=False,
                title='The Statue of Liberty National Monument, New York, USA')
m.add_child(marker)
m

# Нарисуйте на интерактивной карте Нью-Йорка ячейки так, чтобы их цвет показывал среднее за месяц количество поездок такси в час из этой зоны.

In [17]:
pickups = taxi201605_agg.mean(axis=0)
pickups

1       0.0
2       0.0
3       0.0
4       0.0
5       0.0
       ... 
2496    0.0
2497    0.0
2498    0.0
2499    0.0
2500    0.0
Length: 2500, dtype: float64

In [18]:
pickups = pickups[pickups>0]

In [19]:
state_data = pd.DataFrame()
state_data["id"]=pickups.index.values
state_data["counts"] = pickups.values

In [23]:
m = Map(location=(lat, lon), zoom_start=12, no_touch=False, zoom_control=True,
        draggable=True, touch_zoom=True, double_click_zoom=True)

Choropleth(
    geo_data=state_geo,
    name='taxi',
    data=state_data,
    columns=['id', 'counts'],
    key_on='feature.id',
    fill_color='YlOrRd',
    fill_opacity=0.6,
    line_opacity=0.2,
    legend_name="Average"
).add_to(m)
marker = Marker(location=(lat, lon), draggable=False,
                title='The Statue of Liberty National Monument, New York, USA')
marker.add_to(m)
m

**карта показывает, что имеются пикапы в заливе рядом со статуей свободы**

# Чтобы не выбирать из всех 2500 ячеек вручную, отфильтруйте ячейки, из которых в мае совершается в среднем меньше 5 поездок в час. Посчитайте количество оставшихся. Проверьте на карте, что среди этих ячеек нет таких, из которых поездки на самом деле невозможны.

отфильтруйте ячейки, из которых в мае совершается в среднем меньше 5 поездок в час

In [24]:
pickups = pickups[pickups>=5]
pickups

1075     66.799731
1076    138.266129
1077     86.802419
1125     66.041667
1126    181.892473
           ...    
2068     95.056452
2069     10.545699
2118    146.490591
2119     34.692204
2168     68.532258
Length: 102, dtype: float64

Посчитайте количество оставшихся

In [25]:
len(pickups)

102

In [26]:
state_data = pd.DataFrame()
state_data["id"]=pickups.index.values
state_data["counts"] = pickups.values

Проверьте на карте, что среди этих ячеек нет таких, из которых поездки на самом деле невозможны

In [28]:
m = Map(location=(lat, lon), zoom_start=11, no_touch=False, zoom_control=True,
        draggable=True, touch_zoom=True, double_click_zoom=True)

Choropleth(
    geo_data=state_geo,
    name='taxi',
    data=state_data,
    columns=['id', 'counts'],
    key_on='feature.id',
    fill_color='YlOrRd',
    fill_opacity=0.6,
    line_opacity=0.2,
    legend_name="Average"
).add_to(m)
marker = Marker(location=(lat, lon), draggable=False,
                title='The Statue of Liberty National Monument, New York, USA')
marker.add_to(m)
m

**карта очищена от пикапов в заливе**

# Сохраните ноутбук в формате html, запакуйте ipynb и html в один архив и загрузите его в форму (html облегчит задачу проверки вашим рецензентам — в ipynb динамические карты часто не сохраняются, а для выполнения кода рецензентам придётся устанавливать все библиотеки). 